In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

pd.options.display.max_columns = None 
pd.options.display.max_rows = None

import timeit

In [2]:
train = pd.read_csv('../Data/train_clean.csv')

In [3]:
train.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,home_timeouts_remaining,away_timeouts_remaining,timeout,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,return_yards,replay_or_challenge,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,field_goal,extra_point,two_point_conversion
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,900.0,1800.0,3600.0,Half1,0,1,0,1,0.0,0.0,15:00,TEN 30,0,0,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,kickoff,0.0,0,0,0.0,0,0,0,3,3,0.0,3.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.001506,0.179749,0.006639,0.281138,0.213700,0.003592,0.313676,0.0,0.0,0.323526,2.014474,2.014474,-2.014474,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0,0.0,0.0,0.0,0.0,0,0,0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,893.0,1793.0,3593.0,Half1,0,1,0,1,1.0,0.0,14:53,PIT 42,10,5,(14:53) B.Roethlisberger pass short left to H....,pass,5.0,0,0,1.0,0,0,0,3,3,0.0,3.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000969,0.108505,0.001061,0.169117,0.293700,0.003638,0.423011,0.0,0.0,2.338000,0.077907,2.092381,-2.092381,0.00000,0.00000,0.077907,-0.077907,-0.938735,1.016643,-0.938735,0.938735,1.016643,-1.016643,-0.938735,0.938735,1.016643,-1.016643,0.546433,0.453567,0.546433,0.453567,0.004655,0.551088,0.448912,0.000000,0.000000,0.004655,-0.004655,-0.028383,0.033038,-0.028383,0.028383,0.033038,-0.033038,-0.028383,0.028383,0.033038,-0.033038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### Preprocessing and Feature engineering

In [4]:
# adding column for succesful play, will be dependent variable 
train['succesful_play'] = (np.where((train['play_type'] == 'pass')& (train['yards_gained']>= 6.3), 1,
                                   np.where((train['play_type'] == 'run') & (train['yards_gained'] >= 4.4), 1, 0)))

In [5]:
# creating column with concatenated string of home team and away team as game setting
train['game_setting'] = train['home_team'].str.cat(train['away_team'])

In [6]:
# number of unique combinations of home and away team (NFL has 32 teams)
train['game_setting'].nunique()

912

In [7]:
%timeit dummies = pd.get_dummies(train['game_setting'])

133 ms ± 8.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
# create dummy variable columns out of game setting
dummies = pd.get_dummies(train['game_setting'])

In [9]:
# joining train and dummy df
train = train.join(dummies)


In [10]:
# adding dummy column for team in posation for current play 
train['posteam_is_home'] = (np.where(train['posteam_type'] == 'home', 1 , 0))

In [11]:
# adding dummy columns for the type opf play
playtype = pd.get_dummies(train['play_type'], 
                          prefix= 'pt', drop_first = True )

In [12]:
playtype.head()

,pt_extra_point,pt_field_goal,pt_kickoff,pt_no_play,pt_pass,pt_punt,pt_qb_kneel,pt_qb_spike,pt_run
0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0


In [13]:
train = train.join(playtype)

In [14]:
# changing strings in column game_half to integers (Half1 == 1 and Half2 == 2)
train['game_half'] = np.where(train['game_half'] == 'Half1', 1, 2)

In [15]:
train['game_date'] = pd.to_datetime(train['game_date'],  format = '%Y-%m-%d' )

In [16]:
# assigning column to df that replaces data side_of_field 
train['side_of_field_is_hometeam'] = np.where(train['side_of_field'] == train['home_team'], 1, 0)

In [17]:
#train.dtypes


In [18]:
# dropping the columns which are no longer necessary

object_cols = ['home_team','away_team' ,'posteam' ,'posteam_type',
               'defteam', 'side_of_field','time','yrdln', 'desc','play_type', 'game_setting']

train_ready = train.copy()
train_ready = train_ready.drop(columns = object_cols)

### Logistic Regression model

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [20]:
cols = [col for col in train_ready.columns]
cols.remove('succesful_play')
cols.remove('game_date')
cols.remove('yards_gained')

In [21]:
X = train_ready[cols]
y = train_ready['yards_gained']


In [22]:
#logreg = LogisticRegression()
#logreg.fit(X, y.values.ravel())

### Cross Validation

In [ ]:
cross_validate(LogisticRegression(max_iter = 1000), X, y.values.ravel(),
               cv = 5, scoring = 'accuracy')

In [24]:
#models = {'LogReg':LogisticRegression(),
#         'Decision Tree':DecisionTreeClassifier(random_state=42),
#         'Random Forest': RandomForestClassifier(n_estimators=100, n_jobs=-1)}

In [ ]:
#cross_validate(DecisionTreeClassifier(random_state=42), X, y.values.ravel(),
               #cv = 5, scoring = 'accuracy')

In [ ]:
#model_validation(DecisionTreeClassifier(random_state=42))

In [ ]:
#model_validation(RandomForestClassifier(n_estimators=100, n_jobs=-1))

In [ ]:
#model_validation(LogisticRegression())